In [ ]:
import pandas as pd
import tensorflow as tf
from konlpy.tag import Mecab
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import os
from tensorflow import keras

import konlpy
import gensim
from gensim.models import KeyedVectors
from tensorflow.keras.initializers import Constant
from gensim.models.keyedvectors import Word2VecKeyedVectors

print(konlpy.__version__)
print(gensim.__version__)

import pandas as pd

train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')

train_data.head()

from konlpy.tag import Mecab
import numpy as np
from collections import Counter

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    #중복제거
    train_data.drop_duplicates(subset=['document'], inplace=True)
    test_data.drop_duplicates(subset=['document'], inplace=True)
    #결측치제거
    train_data = train_data.dropna(how = 'any') 
    test_data = test_data.dropna(how = 'any')

    X_train = []
    X_test = []

    #토큰화&불용어제거
    for sentence in train_data['document']:
        temp_X = tokenizer.morphs(sentence)
        temp_X = [word for word in temp_X if not word in stopwords]
        X_train.append(temp_X)

    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence)
        temp_X = [word for word in temp_X if not word in stopwords]
        X_test.append(temp_X)
    
    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4) #상위 9996
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter] #인덱스정의
    word_to_index = {word:index for index, word in enumerate(vocab)}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]
    
    #벡터화
    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))

    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index
    
    
x_train, y_train, x_test, y_test, word_to_index = load_data(train_data, test_data)

index_to_word = {index:word for word, index in word_to_index.items()}

# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

#데이터 문장 리스트 생성
total_data_text = list(x_train) + list(x_test)
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)

#최대 길이
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print(f'tolerance: {np.sum(num_tokens < max_tokens) / len(num_tokens)}%')

# Padding seq 생성
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=word_to_index["<PAD>"],
                                                        padding='pre', 
                                                        maxlen=maxlen)

x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value=word_to_index["<PAD>"],
                                                       padding='pre', 
                                                       maxlen=maxlen)

print('pre:' ,x_train.shape)

vocab_size = 10000  #어휘 사전의 크기(10000단어)
word_vector_dim = 16  #워드 벡터의 차원 수

#RNN
rnn_model = tf.keras.Sequential()
rnn_model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
rnn_model.add(tf.keras.layers.LSTM(8))
rnn_model.add(tf.keras.layers.Dense(8, activation='relu'))
rnn_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))#최종 출력은 긍정/부정을 나타내는 1dim

rnn_model.summary()

#1-D CNN
cnn_model = tf.keras.Sequential()
cnn_model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
cnn_model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
cnn_model.add(tf.keras.layers.MaxPooling1D(5))
cnn_model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
cnn_model.add(tf.keras.layers.GlobalMaxPooling1D())
cnn_model.add(tf.keras.layers.Dense(8, activation='relu'))
cnn_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

cnn_model.summary()

#GlobalMaxPooling1D
gloMP_model = tf.keras.Sequential()
gloMP_model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
gloMP_model.add(tf.keras.layers.GlobalMaxPooling1D())
gloMP_model.add(tf.keras.layers.Dense(8, activation='relu'))
gloMP_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

gloMP_model.summary()

#validation set 10000건 분리
x_val = x_train[:10000]   
y_val = y_train[:10000]

#나머지
partial_x_train = x_train[10000:]  
partial_y_train = y_train[10000:]

print(partial_x_train.shape)
print(partial_y_train.shape)

epochs=10 

#RNN
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_history = rnn_model.fit(partial_x_train, partial_y_train, epochs=epochs,
                    batch_size=512, validation_data=(x_val, y_val), verbose=1)
#CNN
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_history = cnn_model.fit(partial_x_train, partial_y_train, epochs=epochs,
                    batch_size=512, validation_data=(x_val, y_val), verbose=1)
#GlobalMaxPooling1D
gloMP_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gloMP_history = gloMP_model.fit(partial_x_train, partial_y_train, epochs=epochs,
                    batch_size=512, validation_data=(x_val, y_val), verbose=1)

# 테스트셋으로 평가
rnn_results = rnn_model.evaluate(x_test,  y_test, verbose=2)
cnn_results = cnn_model.evaluate(x_test,  y_test, verbose=2)
gloMP_results = gloMP_model.evaluate(x_test,  y_test, verbose=2)

print('='*70)
print('평가 결과')
print('RNN : ', rnn_results)
print('CNN : ', cnn_results)
print('GlobalMaxPooling1D : ', gloMP_results)
print('='*70)

#그래프
rnn_history_dict = rnn_history.history
cnn_history_dict = cnn_history.history
gloMP_history_dict = gloMP_history.history

# RNN
rnn_acc = rnn_history_dict['accuracy']
rnn_val_acc = rnn_history_dict['val_accuracy']
rnn_loss = rnn_history_dict['loss']
rnn_val_loss = rnn_history_dict['val_loss']

# 1-D CNN
cnn_acc = cnn_history_dict['accuracy']
cnn_val_acc = cnn_history_dict['val_accuracy']
cnn_loss = cnn_history_dict['loss']
cnn_val_loss = cnn_history_dict['val_loss']

# GlobalMaxPooling1D
gloMP_acc = gloMP_history_dict['accuracy']
gloMP_val_acc = gloMP_history_dict['val_accuracy']
gloMP_loss = gloMP_history_dict['loss']
gloMP_val_loss = gloMP_history_dict['val_loss']

# Training and validation loss
fig, ax = plt.subplots(1, 3, figsize=(20, 10))

# RNN
rnn_epochs = range(1, len(rnn_acc) + 1)
ax[0].plot(rnn_epochs, rnn_loss, 'bo', label='RNN Training loss')
ax[0].plot(rnn_epochs, rnn_val_loss, 'b', label='RNN Validation loss')
# 1-D CNN
cnn_epochs = range(1, len(rnn_acc) + 1)
ax[1].plot(cnn_epochs, cnn_loss, 'bo', label='CNN Training loss')
ax[1].plot(cnn_epochs, cnn_val_loss, 'b', label='CNN Validation loss')
# GlobalMaxPooling1D
gloMP_epochs = range(1, len(rnn_acc) + 1)
ax[2].plot(gloMP_epochs, gloMP_loss, 'bo', label='GloMP Training loss')
ax[2].plot(gloMP_epochs, gloMP_val_loss, 'b', label='GloMP Validation loss')

plt.show()

# Training and validation accuracy
# 그림 초기화
plt.clf()   
# Training and validation loss
fig, ax = plt.subplots(1, 3, figsize=(20, 10))

# RNN
ax[0].plot(rnn_epochs, rnn_acc, 'bo', label='RNN Training acc')
ax[0].plot(rnn_epochs, rnn_val_acc, 'b', label='RNN Validation acc')
# 1-D CNN
ax[1].plot(cnn_epochs, cnn_acc, 'bo', label='CNN Training acc')
ax[1].plot(cnn_epochs, cnn_val_acc, 'b', label='CNN Validation acc')
# GlobalMaxPooling1D
ax[2].plot(gloMP_epochs, gloMP_acc, 'bo', label='GloMP Training acc')
ax[2].plot(gloMP_epochs, gloMP_val_acc, 'b', label='GloMP Validation acc')

plt.show()

#Embedding 레이어 분석
model = rnn_model
embedding_layer = model.layers[0]
weights = embedding_layer.get_weights()[0]
print(weights.shape)

word2vec_file_path = os.getenv('HOME')+'/aiffel/sentiment_classification/data/ex04.txt'
f = open(word2vec_file_path, 'w')
f.write('{} {}\n'.format(vocab_size-4, word_vector_dim))

# 단어 개수만큼 워드 벡터를 파일에 기록
vectors = model.get_weights()[0]
for i in range(4,vocab_size):
    f.write('{} {}\n'.format(index_to_word[i], ' '.join(map(str, list(vectors[i, :])))))
f.close()

word_vectors = Word2VecKeyedVectors.load_word2vec_format(word2vec_file_path, binary=False)
vector = word_vectors['끝']
vector


word_vectors.similar_by_word("끝")

#한국어 Word2Vec 임베딩
word2vec_file_path = os.getenv('HOME')+'/data/word2vec_ko.model'
word2vec = gensim.models.Word2Vec.load(word2vec_file_path)
word2vec.wv.similar_by_word("끝")

vocab_size = 10000    
word_vector_dim = 100  

embedding_matrix = np.random.rand(vocab_size, word_vector_dim)

for i in range(4,vocab_size):
    if index_to_word[i] in word2vec.wv:
        embedding_matrix[i] = word2vec.wv[index_to_word[i]]
vocab_size = 10000    
word_vector_dim = 100  

model = tf.keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 
                                 word_vector_dim, 
                                 embeddings_initializer=Constant(embedding_matrix),  #카피한 임베딩을 여기서 활용
                                 input_length=maxlen, 
                                 trainable=True))   #trainable을 True로 주면 Fine-tuning
model.add(keras.layers.LSTM(128))
rnn_model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

results = model.evaluate(x_test,  y_test, verbose=2)

print(results)